In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=0e2610ef5f2049e6e027d06ec53aa99289563e030844ce8fb4aeb9e50c5ed011
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark=SparkSession.builder.appName("stocks project").getOrCreate()
df=spark.read.csv("/content/cleaned_data.csv",header=True,inferSchema=True)
df.printSchema()
df.write.format("delta").mode("overwrite").save("/content/cleaned_data_delta")
delta_df=spark.read.format("delta").load("/content/cleaned_data_delta")

delta_table=spark.readTable("cleaned_data_delta")
delta_table.printSchema()
delta_table.show()
spark.sql("optimize cleaned_data_delta")
spark.sql("vacuum cleaned_data_delta retain 168 hours")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("stocks project") \
    .getOrCreate()

# Read streaming data from a socket (for demonstration purposes)
df_stream = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .schema("Stock_id INT, Company_id INT, Stock_Price FLOAT, Trading_Volume INT, Moving_Average FLOAT, Market_Cap FLOAT, RSI FLOAT, Record_Time TIMESTAMP") \
    .load("/content/cleaned_data.csv")

query=df_stream.writeStream.format("console").outputMode("append").start()
# Calculate the average of Moving_average
# Filter the anomalies
anomalies = df_stream.filter(
    (F.col('Market_Cap') < 10) |
    (F.col("trading_volume") > 1000000)
)
anomalies.printSchema()
query2=anomalies.writeStream.format("console").outputMode("append").start()